In [1]:
import pandas as pd
import requests
import json
import glob
import itertools
import re

In [2]:
from sqlalchemy import create_engine

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning) 

In [5]:
url = 'https://www.dooinauction.com/auction/search/?km_status=ing&current_page=1&records_per_page=100'
driver = webdriver.Chrome(executable_path='chromedriver.exe')
driver.get(url)
driver.implicitly_wait(30)
soup = BeautifulSoup(driver.page_source, 'lxml')
driver.close()
last = int(soup.find_all('a','page-link')[-2].text)
print(last)

118


In [18]:
data = pd.DataFrame()

for p in range(last):
    p+=1
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    url = f'https://www.dooinauction.com/auction/search/?km_status=ing&current_page={p}&records_per_page=100'
    # url = f'https://www.dooinauction.com/auction/search/?km_status=ing&addr1%5B%5D=%EC%84%9C%EC%9A%B8&addr1%5B%5D=%EA%B2%BD%EA%B8%B0&current_page={p}&records_per_page=100'
    driver.get(url)
    print(url)
    driver.implicitly_wait(30)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    items = soup.find_all("div","list-product")
    # if len(items) ==0:
    #     print("BREAK")
    #     break
        
    # driver.close()
    df = pd.DataFrame()
    for i in items:
        d = pd.DataFrame()
        one = i.find_all('span',{'address-txt'})
        d['경매번호'] = [one[0].text.replace('지도 보기 ','')]
        d['주소지'] = [one[1].text]
        try:
            d['x'] = [(str(one[0]).split('urlX=')[1].split('&')[0])]
            d['y'] = [(str(one[0]).split('urlY=')[1].split('"')[0])]
        except IndexError:
            d['x'] = [""]
            d['y'] = [""]
        except ValueError:
            d['x'] = [""]
            d['y'] = [""]
        two = i.find_all('div',{'court-txt'})
        d['종류']  = [two[0].text]
        try:
            three = i.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['size-txt'])
            d['면적'] = [three[0].text]
        except IndexError:
            d['면적'] = ['']
        
        four = i.find_all('div','price-value')
        try:
            d['감정가'] = [int(four[0].text.replace(',',""))]
            d['입찰가'] = [int(four[1].text.replace(',',""))]
        except IndexError:
            d['감정가'] = [0]
            d['입찰가'] = [0]
        
        five = i.find_all('div', 'date-day')        
        d['입찰일'] = [five[0].text]
        
        six = i.find_all('input',value=True)[0]['value']
        try:
            d['링크'] = [six]
        except IndexError:
            d['링크'] = [0]
        df = df.append(d)
        
    data = data.append(df)
    

https://www.dooinauction.com/auction/search/?km_status=ing&current_page=1&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=2&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=3&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=4&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=5&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=6&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=7&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=8&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=9&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=10&records_per_page=100
https://www.dooinauction.com/auction/search/?km_s

In [13]:
# data['감정가'] = data['감정가'].str.replace(',','')

In [19]:
data['크기'] = 10

data['x'] = pd.to_numeric(data['x'],errors='coerce')
data['y'] = pd.to_numeric(data['y'],errors='coerce')

# data['감정가']=data['감정가'].apply(lambda x : "{:,}".format(x))
# data['입찰가']=data['입찰가'].apply(lambda x : "{:,}".format(x))

data['감정가'] = pd.to_numeric(data['감정가'],errors='coerce')
data['입찰가'] = pd.to_numeric(data['입찰가'],errors='coerce')

data['감정가']=data['감정가'].apply("{:,}".format)
data['입찰가']=data['입찰가'].apply("{:,}".format)

data['링크'] = data["링크"].apply(lambda x: '<a href="https://www.dooinauction.com/auction/search/?auction_id={0}">link</a>'.format(x))

In [23]:
data.to_excel('data.xls',index=False)

In [30]:
data.to_csv("data.csv",encoding ='euc_kr',index=False)

In [29]:
pd.read_excel('https://github.com/yundk7/dooin/raw/main/data.xls')

,경매번호,주소지,x,y,종류,면적,감정가,입찰가,입찰일,링크,크기
0,2020 타경 1096(1),경북 김천시 부곡동 619,128.096071,36.121751,전,토지 1302㎡(393.86평),"158,844,000","54,484,000",2022-12-16,"<a href=""https://www.dooinauction.com/auction/...",10
1,2020 타경 1279,경북 영덕군 남정면 원척리 268 외 3개 목록,129.376972,36.296550,주유소,건물 406.76㎡(123.04평) / 토지 1782㎡(539.06평),"972,288,960","680,602,000",2022-12-16,"<a href=""https://www.dooinauction.com/auction/...",10
2,2020 타경 2969,부산 동래구 복천동 500-1 우성베스토피아 4동 2층 205호,129.088038,35.206843,아파트,건물 124.7㎡(37.72평) / 대지권 56.1㎡(16.97평),"336,000,000","336,000,000",2022-12-16,"<a href=""https://www.dooinauction.com/auction/...",10
3,2020 타경 11153(1),경북 울진군 울진읍 호월리 1043,129.354750,37.009311,답,토지 74㎡(22.39평),"244,200","20,000",2022-12-16,"<a href=""https://www.dooinauction.com/auction/...",10
4,2020 타경 28766,경남 거제시 고현동 41-8 중앙빌딩 비동 2층 209호 외 1개 목록,128.625235,34.888013,근린상가,건물 84.24㎡(25.48평) / 대지권 34㎡(10.29평),"300,000,000","147,000,000",2022-12-16,"<a href=""https://www.dooinauction.com/auction/...",10
...,...,...,...,...,...,...,...,...,...,...,...
11542,2022 타경 54485,전남 무안군 일로읍 지장리 535-1 내(김형채 ***-****-****) 보관,NaN,NaN,차량,"수입차, 푸조 308 SW 1.6 e-HDi2014년식 172,851㎞ 자동 경유","9,000,000","6,300,000",2023-02-13,"<a href=""https://www.dooinauction.com/auction/...",10
11543,2021 타경 35971,대구 달서구 도원동 1443 대곡강산타운 401동 19층 1903호,128.536593,35.805386,아파트,건물 42.49㎡(12.85평) / 대지권 19.03㎡(5.76평),"194,000,000","95,060,000",2023-02-14,"<a href=""https://www.dooinauction.com/auction/...",10
11544,2022 타경 1235,전남 영광군 대마면 송죽리 1029-10 외 2개 목록,126.571078,35.287934,공장,건물 1452.9㎡(439.5평) / 토지 3305.3㎡(999.85평),"1,397,916,800","978,542,000",2023-02-22,"<a href=""https://www.dooinauction.com/auction/...",10
11545,2021 타경 6776(1),전남 나주시 빛가람동 186-1 바동 1층 101호 외 6개 목록,126.781040,35.024330,근린상가,건물 931.5㎡(281.78평) / 대지권 613.1㎡(185.46평),"5,475,000,000","3,066,000,000",2023-03-02,"<a href=""https://www.dooinauction.com/auction/...",10
